Currently I 'm still modify the file.

This analysis is based on several articles and I will write the references later. I'm writing a calculator based on Canada's Covid-19 analysis, especially masks and social distances, and use this for student's Covid-19 website project. Currently there is no machine learning.

# Catalog:

1.basic info
2.mask
3.social distance
4.variable name
5.

# Basic info and data

In [ ]:
#import modules
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
#read data, world Covid-19 datasets
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [ ]:
df_confirmed.head()

In [ ]:
#select Canada data
df_confirmed_Canada = df_confirmed.loc[df_confirmed['Country/Region'] == 'Canada'].reset_index(drop = True)
df_deaths_Canada = df_deaths.loc[df_confirmed['Country/Region'] == 'Canada'].reset_index(drop = True)
df_recovered_Canada = df_recovered.loc[df_confirmed['Country/Region'] == 'Canada'].reset_index(drop = True)

In [ ]:
#calculate sums in last row
#Canada_disease_trend = pd.DataFrame(np.zeros((1, df_confirmed_Canada.shape[1])), columns = df_confirmed_Canada.columns)
#Canada_disease_trend
df_confirmed_Canada = df_confirmed_Canada.append(df_confirmed_Canada.sum(axis=0).to_frame().transpose(), ignore_index = True)
df_deaths_Canada = df_deaths_Canada.append(df_deaths_Canada.sum(axis=0).to_frame().transpose(), ignore_index = True)
df_recovered_Canada = df_recovered_Canada.append(df_recovered_Canada.sum(axis=0).to_frame().transpose(), ignore_index = True)

In [ ]:
#change last row name to total, 
df_confirmed_Canada['Province/State'][len(df_confirmed_Canada['Province/State'])-1] = 'total'
df_deaths_Canada['Province/State'][len(df_deaths_Canada['Province/State'])-1] = 'total'
df_recovered_Canada['Province/State'][len(df_recovered_Canada['Province/State'])-1] = 'total'

#drop useless columns
df_confirmed_Canada.drop(['Country/Region', 'Lat', 'Long'], axis = 1, inplace = True)
df_deaths_Canada.drop(['Country/Region', 'Lat', 'Long'], axis = 1, inplace = True)
df_recovered_Canada.drop(['Country/Region', 'Lat', 'Long'], axis = 1, inplace = True)

df_deaths_Canada

In [ ]:
#last day's total data, like total death in last day
confirm_today = df_confirmed_Canada.iloc[-1,:][-1]
death_today = df_deaths_Canada.iloc[-1,:][-1]
recover_today = df_recovered_Canada.iloc[-1,:][-1]
print('Confirm, death, recover number in total: ', confirm_today, ',', death_today, ',', recover_today)

In [ ]:
#2 weeks before last day
confirm_2wbefore = df_confirmed_Canada.iloc[-1,:][-14]
death_2wbefore = df_deaths_Canada.iloc[-1,:][-14]
recover_2wbefore = df_recovered_Canada.iloc[-1,:][-14]

In [ ]:
#total num within nearby two weeks
confirm_2w = confirm_today - confirm_2wbefore
death_2w = death_today - death_2wbefore
recover_2w = recover_today - recover_2wbefore
print('Confirm, death, recover number in 2 weeks: ', confirm_2w, ',', death_2w, ',', recover_2w)

In [ ]:
#total Canada population
#k1k1k1 = pd.read_csv('https://raw.githubusercontent.com/ishaberry/Covid19Canada/master/cases.csv')
#k1k1k1.to_csv("../../kaggle/working/Canada_disease.csv", index=False)

New dataset with every CA cases info

In [ ]:
#total Canada disease
Cases_Canada = pd.read_csv('https://raw.githubusercontent.com/ishaberry/Covid19Canada/master/cases.csv')
Deaths_Canada = pd.read_csv('https://raw.githubusercontent.com/ishaberry/Covid19Canada/master/mortality.csv')
test_Canada = pd.read_csv('https://raw.githubusercontent.com/ishaberry/Covid19Canada/master/testing_cumulative.csv')
print(Cases_Canada.shape, Cases_Canada.columns.tolist())

In [ ]:
Deaths_Canada.head()

case_id: number of cases in Canada

provincial_case_id: number of cases in each provinces

when travel_yn is 0: no travel history.
when travel_yn is 1; has travel history

basic info: age, sex, health_region, province, country, report_week, travel_yn, travel_history_country

In [ ]:
#print(Cases_Canada.travel_history_country.unique())

I think age, sex, health_region, province, report_week, travel_yn, travel_history_country is important.

# Mask analysis

Homemade masks: 58% - 77%, Surgical masks: 72% - 85%, N95 and equivalents: 98-99%, 20% effective masks: 5.4% reduces (death rate), 50% effective masks: 31.1% reduces, 80% effective masks: 64.5% reduces


In [ ]:
#efficiency mask parameters %
homemade_mask_E = [58, (58+77)/2, 77]
surgical_mask_E = [72, (72+85)/2, 85]
N95_mask_E = [98, 98.5, 99]

#death rate VS efficiency
efficiency = [20, 50, 80]
death_rate = [5.4, 31.1, 64.5]

Since efficiency and death_rate looks linear, use linear regression. Here I use simple mathematics

I use this caluclator: https://www.analyzemath.com/parabola/three_points_para_calc.html

In [ ]:
#function of efficiency and death rate
E_D_func = lambda efficiency: (77/18000 * efficiency ** 2 + 1003/1800 * efficiency - 671/90) / 100

We have three choices for type of masks and wearing masks

type: homemade, surgical, N95

wearing masks: 20% is seldom, 50% is normal, 80% is always, 100% is every time

solution_optimal: worst, normal, best solutions.

In [ ]:
type_mask_list = ['homemade', 'surgical', 'N95']
wear_mask_list_choice = ['seldom', 'normal', 'always', 'every time']
wear_mask_list = [.2, .5, .8, 1]
wear_mask_dic = {wear_mask_list_choice[i]: wear_mask_list[i] for i in range(len(wear_mask_list_choice))}
#print(wear_mask_dic)

solution_optimal_list = ['worst', 'normal', 'best']
solution_optimal_loc = [0,1,2]
solution_optimal_dic = {solution_optimal_list[i]: solution_optimal_loc[i] for i in range(len(solution_optimal_list))}
#print(solution_optimal_dic)

def mask_efficiency(type_mask, solution_optimal):
    
    #masks efficiency
    homemade_mask_E = [58, (58+77)/2, 77]
    surgical_mask_E = [72, (72+85)/2, 85]
    N95_mask_E = [98, 98.5, 99]

    if type_mask == 'homemade':
        return homemade_mask_E[solution_optimal_dic[solution_optimal]]

    if type_mask == 'surgical':
        return surgical_mask_E[solution_optimal_dic[solution_optimal]]
    
    if type_mask == 'N95':
        return N95_mask_E[solution_optimal_dic[solution_optimal]]
        
#print(mask_efficiency('homemade', 'normal'))

def wear_mask_func(wear_mask):
    if wear_mask == 'seldom':
        return wear_mask_list[0]

    if wear_mask == 'normal':
        return wear_mask_list[1]
    
    if wear_mask == 'always':
        return wear_mask_list[2]
    
    if wear_mask == 'every time':
        return wear_mask_list[3]

#function of our solution
#the third parameter is function of efficiency and death rate
def mask(type_mask, wear_mask, solution_optimal, E_D_func = E_D_func):
    # input: masks information and function 
    # input example: N95, 20%, best, E_D_func
    # output: solution to customers
    
    #error
    if solution_optimal not in solution_optimal_list or type_mask not in type_mask_list or wear_mask not in wear_mask_list_choice:
        print ('Raise error')
        return None
    
    #masks efficiency
    mask_eff = mask_efficiency(type_mask, solution_optimal)
    
    death_rate_reduction = E_D_func(mask_eff * wear_mask_func(wear_mask))
    #print('The reduction of death rate is {} for {} masks, for people who {} wear masks. This solution is the {} choice.'.format(round(death_rate,2), type_mask, wear_mask, solution_optimal))
    #print('By this solution, in Canada, there is {} reduction of people''s death due to the whole Covid-19, {} reduction of people''s death during the last two weeks.'.format(round(death_today * death_rate, 0), round(death_2w * death_rate, 0)))
    
    return death_rate_reduction

#testing
mask('N95', 'always', 'best')
#print(calculator('homemade', 'always', 'normal'))

# Social distance analysis

Here is actions that relate to social distance. User can click the tool box, then 

Stay at home: - 45%

Working full time: + 85%

Close School: - 60%

Distance Travelled from home: + 25%


In [ ]:
social_distance_choice = ['Stay at home', 'Working full time', 'Close School', 'Distance Travelled from home']
social_distance_answer = [-.45, .85, -.60, .25]
social_distance_dic = {social_distance_choice[i]: social_distance_answer[i] for i in range(len(social_distance_choice))}

def social_distance(death_rate, choice_list = [False, False, False, False]):
    #input: death rate, choice(e.g: [True, True, False, True])
    #output: death rate after analysis
    for choice_pos in range(len(choice_list)):
        if choice_list[choice_pos] == True:
            death_rate = death_rate * (1 + social_distance_answer[choice_pos])
    return death_rate 
    
social_distance(1, [True, True, False, True])    

# Other factors

In [ ]:
dt_factors = pd.read_csv('../input/canada-covid19-risk-factors/risk factor.csv')
dt_factors.head()

In [ ]:
#correction of data
dt_factors['Unnamed: 0'][1] = 'Total'
dt_factors['Unnamed: 0'][0] = 'Column_name'

re.compile method

https://www.tutorialspoint.com/Why-do-we-use-re-compile-method-in-Python-regular-expression#:~:text=The%20re.&text=compile(pattern%2C%20repl%2C%20string,pattern%20again%20without%20rewriting%20it.

In [ ]:
import re
pattern = re.compile(r'\d+')
result = int(''.join(pattern.findall('N=5,416')))
print(result)

In [ ]:
for name in dt_factors.columns.tolist()[1:]:
    dt_factors[name][1] = int(''.join(pattern.findall(str(dt_factors[name][1]))))
dt_factors.head()

In [ ]:
#fill NAN
dt_factors = dt_factors.fillna(0)
dt_factors

Hypertension:     The population prevalence of hypertension among Canadians aged 20 to 79 was found to be significantly higher for men (24.5%, 95% CI: 22.7% to 26.4%) then for women (21.5%, 95% CI: 19.8% to 23.2%). 

Coronary artery disease:   According to the most recent data from 2012/13, about 2.4 million (8.5%) Canadian adults aged 20 years and older live with diagnosed ischemic heart disease, including 578,000 (2.1%) with a history of a heart attack.

History of stroke:  The age-standardized occurrence of stroke slightly increased from 2.3% for both sexes in 2003–2004 up to 2.6% in 2012–2013. This represents an average annual increase of 1.0%. During the same time period, the age-standardized occurrence rate of first stroke decreased on average 2.7% annually from 383.0 per 100,000 in 2003–2004 to 297.3 per 100,000 in 2012/13.

Diabetes:  According to the most recent data (Box 1), about 3.0 million Canadians (8.1%) were living with diagnosed diabetes in 2013–2014, representing 1 in 300 children and youth (1–19 years), and 1 in 10 adults (20 years and older)

Obesity: Obesity rate
Public Health of Canada has reported that in 2017, 64% of Canadians over the age of 18 are overweight or obese, and about 30% of children aged 5-17 are overweight or obese. ... The Northwest Territories have the highest obesity rate, at 33.7%.

CKD: CKD is a major public health burden both nationally and internationally. The global mean prevalence of CKD is 13.4%, and within Canada, most recent estimates suggest at least 4 million people have the disease.

Asthma: In 2014, 8.1% of Canadians aged 12 and older, roughly 2.4 million people, reported that they had been diagnosed with asthma by a health professional.

COPD:
In 2019, around 4 percent of Canadians reported that they had been diagnosed by a health professional as having COPD.


By the researches above, we choose Hypertension: 24.5% for men, 21.5% for women

Coronary artery disease: 8.5%

History of Stroke: choose 2.6%

Diabets: now just choose 8.1%, could be more accurate

Obesity: 64%

Chronic Kidney Disease: 13.4%	

Asthma: 8.1%

COPD: 4%

In [ ]:
researched_name = ['Column_name', 'Total','Hypertension','Coronary Artery Disease', 'History of Stroke', 'Diabets', 'Obesity', 'Chronic Kidney Disease', 'Asthma ', 'COPD']
researched_data = [0,0,.23,.085,.026,.081,.64,.134,.81,.04]
researched_dic = {researched_name[i]: researched_data[i] for i in range(len(researched_name))}
dt_factors_researched = dt_factors.copy()

dt_factors_researched = dt_factors_researched.loc[dt_factors_researched['Unnamed: 0'].isin(researched_name)]
dt_factors_researched    

Users can choose disease, age, sex, race groups (default is not answer these questions) in the future.

But for now, we only have the data of total number.

for this document, user can only choose disease. That means, we only use column 0 and 1

I will use the rate of people who infected, divided by people not infected rate.

In [ ]:
def other_factor(dt_factors_researched, disease = None, age = None, sex = None, race = None):
    if disease is None:
        return 1
    infect_rate_infected = int(dt_factors_researched.loc[dt_factors_researched['Unnamed: 0'] == disease]['Unnamed: 1']) / 5416
    #print('infect rate infected:', infect_rate_infected)
    infect_rate_notinfected = researched_dic[disease]
    #print('infect rate notinfected:', infect_rate_notinfected)
    infect_rate_increase_rate = infect_rate_infected / infect_rate_notinfected
    return infect_rate_increase_rate

for name in researched_name[2:]:
    print(other_factor(dt_factors_researched, name))

In [ ]:
dt_factors_researched.loc[dt_factors_researched['Unnamed: 0'] == 'History of Stroke']['Unnamed: 1']

# final calculator：

Since our solution is compare with no protection with mask and social distancing and caluclate the reduction of death rate. We need to know how Canada did right now. What is the approximate percentage Canada decrease the death rate?
Just like, Canada already have social distancing and masks regulations and laws. How much still can be improved?

The default is Canada reduced 50% of deathes.

# comment:
# type_mask: 'homemade', 'surgical', 'N95'
# wear_mask: 'seldom', 'normal', 'always', 'every time'
# solution_optimal: 'worst', 'normal', 'best'
# choice_list: [False, False, False, False] or [True, False, True, False]

In [ ]:
def calculator(type_mask, wear_mask, solution_optimal, choice_list = [False, False, False, False], dt_factors_researched = dt_factors_researched, E_D_func = E_D_func, disease = None):

    default_reduction = .5 #suppose currently, how much death rate reduction Canada already did
    death_rate_mask = mask(type_mask, wear_mask, solution_optimal, E_D_func = E_D_func)
    death_rate_SD = social_distance(death_rate_mask, choice_list)
    death_rate_final = death_rate_SD * other_factor(dt_factors_researched, disease) - default_reduction
    print('The reduction of death rate is {}% for {} masks, for people who {} wear masks. This solution is the {} choice.'.format(round(death_rate_final,2)*100, type_mask, wear_mask, solution_optimal))
    print('By this solution, in Canada, there is {} reduction of people''s death due to the whole Covid-19, {} reduction of people''s death during the last two weeks.'.format(round(death_today * death_rate_final, 0), round(death_2w * death_rate_final, 0)))
    
    return death_rate_final
    
    
calculator('homemade', 'always', 'normal', [True, True, False, True], disease = 'History of Stroke')    
calculator('homemade', 'seldom', 'worst')   

Parameters:

total number of disease: confirm_today, death_today, recover_today

two-week disease: confirm_2w, death_2w, recover_2w

function of efficiency and death rate: E_D_func

possible choices: type_mask_list, wear_mask_list_choice, solution_optimal_list
